# In this notebook I will calculate the average correlation through all runs

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
import numpy as np
import xesmf as xe
from   scipy.interpolate import griddata
import dask as dsk

import hvplot.xarray
import hvplot.pandas

from   cartopy import config
import cartopy.crs as ccrs
proj = ccrs.PlateCarree()

from scipy import stats
from scipy.stats import t

In [2]:
# Upload all the runs:
run_12= xr.open_dataset("/nobackup/users/ommeren/ECEarth/1-2_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_34= xr.open_dataset("/nobackup/users/ommeren/ECEarth/3-4_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_56= xr.open_dataset("/nobackup/users/ommeren/ECEarth/5-6_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_78= xr.open_dataset("/nobackup/users/ommeren/ECEarth/7-8_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_910= xr.open_dataset("/nobackup/users/ommeren/ECEarth/9-10_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_1112= xr.open_dataset("/nobackup/users/ommeren/ECEarth/11-12_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_1314= xr.open_dataset("/nobackup/users/ommeren/ECEarth/corr_13-14_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")
run_1516= xr.open_dataset("/nobackup/users/ommeren/ECEarth/15-16_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc")

In [3]:
run_12

<xarray.Dataset>
Dimensions:             (lev: 38, run: 2, lat: 180, lon: 170, time: 165,
                         vertices: 4)
Coordinates:
  * lev                 (lev) float64 1.556 2.668 3.856 ... 370.7 411.8 457.6
  * run                 (run) object 'r1i1p5f1' 'r2i1p5f1'
  * lat                 (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float64 120.5 121.5 122.5 ... 287.5 288.5 289.5
  * time                (time) datetime64[ns] 1850-12-31 ... 2014-12-31
Dimensions without coordinates: vertices
Data variables:
    vertices_latitude   (time, run, vertices, lat, lon, lev) float64 ...
    vertices_longitude  (time, run, vertices, lat, lon, lev) float64 ...
    thetao              (time, run, lev, lat, lon) float32 ...

In [4]:
run_1 = run_12.sel(run="r2i1p5f1")
run_1

<xarray.Dataset>
Dimensions:             (lev: 38, lat: 180, lon: 170, time: 165, vertices: 4)
Coordinates:
  * lev                 (lev) float64 1.556 2.668 3.856 ... 370.7 411.8 457.6
    run                 <U8 'r2i1p5f1'
  * lat                 (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float64 120.5 121.5 122.5 ... 287.5 288.5 289.5
  * time                (time) datetime64[ns] 1850-12-31 ... 2014-12-31
Dimensions without coordinates: vertices
Data variables:
    vertices_latitude   (time, vertices, lat, lon, lev) float64 ...
    vertices_longitude  (time, vertices, lat, lon, lev) float64 ...
    thetao              (time, lev, lat, lon) float32 ...

In [5]:
run_1314

<xarray.Dataset>
Dimensions:             (lev: 38, run: 2, lat: 180, lon: 170, time: 165,
                         vertices: 4)
Coordinates:
  * lev                 (lev) float64 1.556 2.668 3.856 ... 370.7 411.8 457.6
  * run                 (run) object 'r13i1p5f1' 'r14i1p5f1'
  * lat                 (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float64 120.5 121.5 122.5 ... 287.5 288.5 289.5
  * time                (time) datetime64[ns] 1850-12-31 ... 2014-12-31
Dimensions without coordinates: vertices
Data variables:
    vertices_latitude   (time, run, vertices, lat, lon, lev) float64 ...
    vertices_longitude  (time, run, vertices, lat, lon, lev) float64 ...
    thetao              (time, run, lev, lat, lon) float32 ...

In [6]:
run_13 = run_1314.sel(run="r14i1p5f1")
run_13

<xarray.Dataset>
Dimensions:             (lev: 38, lat: 180, lon: 170, time: 165, vertices: 4)
Coordinates:
  * lev                 (lev) float64 1.556 2.668 3.856 ... 370.7 411.8 457.6
    run                 <U9 'r14i1p5f1'
  * lat                 (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float64 120.5 121.5 122.5 ... 287.5 288.5 289.5
  * time                (time) datetime64[ns] 1850-12-31 ... 2014-12-31
Dimensions without coordinates: vertices
Data variables:
    vertices_latitude   (time, vertices, lat, lon, lev) float64 ...
    vertices_longitude  (time, vertices, lat, lon, lev) float64 ...
    thetao              (time, lev, lat, lon) float32 ...

### Definitions 

In [3]:
def get_begin_path(run_data, lat_range = slice(-28, -23), lon_range = slice(263, 282)):
    depth_level = 0

    # Get depth closest to 0 m 
    depth_idx = abs(run_data.lev - depth_level).argmin().item()

    # Select region and depth
    begin_path = run_data.thetao.sel(lat=lat_range, lon=lon_range).isel(lev=depth_idx)

    # Average over latitude and longitude
    begin_path_avg_0m = begin_path.mean(dim=["lat", "lon"])

    return begin_path_avg_0m

### Names

In [4]:
# File path structure:
path = "/nobackup/users/ommeren/ECEarth"
name = "_EC-Earth_full_pacific_ocean_temperature_yearly_anomalies.nc"

# Name the runs:
runs = ["1-2", "3-4", "5-6", "7-8", "9-10", "11-12", "corr_13-14", "15-16"]

#Specify the lags:
lags = range(-10, 20)

### Time Lag Correlation

In [5]:
### Loop that includes lag = 

# Define the lags to analyze
lags = range(-3, 20)  

list_of_xarrays = []

for run_pair in runs:
    
    ds = xr.open_dataset(f"{path}/{run_pair}{name}")

       
    for single_run in ds.run:
        print(single_run)
        
        run_data = ds.sel(run=single_run)

        begin_path_avg_0m = get_begin_path(run_data)

        # Here i would just adjust the lev = depth to get the avg corr for different depths 
        section_0m = run_data.thetao.sel(lev = 150, method = "nearest")

        correlation_sec_0m = []

        for lag in lags:
            if lag == 0:
                # For Lag 0, do not shift data
                lag_corr_0m = xr.corr(
                    begin_path_avg_0m,  # No time slicing
                    section_0m,#.sel(lat=slice(-40, 40)),  # No shifting for Lag 0
                    dim="time"
                )
            else: 
                #Shift Section in time
                shifted_section = section_0m.shift(time=-lag).isel(time=slice(None, -abs(lag)))#.sel(lat=slice(-40,40))
    
                # Calculate correlation for the current lag
                lag_corr_0m = xr.corr(
                    begin_path_avg_0m.isel(time=slice(abs(lag), None)),
                    shifted_section,
                    dim="time"
                )

            correlation_sec_0m.append(lag_corr_0m.assign_coords(lag=lag))
            
        correlation_0m = xr.concat(correlation_sec_0m, dim="lag")
        list_of_xarrays.append(correlation_0m)

xr_output = xr.concat(list_of_xarrays, dim="run")

<xarray.DataArray 'run' ()>
array('r1i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r1i1p5f1'
<xarray.DataArray 'run' ()>
array('r2i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r2i1p5f1'
<xarray.DataArray 'run' ()>
array('r3i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r3i1p5f1'
<xarray.DataArray 'run' ()>
array('r4i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r4i1p5f1'
<xarray.DataArray 'run' ()>
array('r5i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r5i1p5f1'
<xarray.DataArray 'run' ()>
array('r6i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r6i1p5f1'
<xarray.DataArray 'run' ()>
array('r7i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r7i1p5f1'
<xarray.DataArray 'run' ()>
array('r8i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r8i1p5f1'
<xarray.DataArray 'run' ()>
array('r9i1p5f1', dtype='<U8')
Coordinates:
    run      <U8 'r9i1p5f1'
<xarray.DataArray 'run' ()>
array('r10i1p5f1', dtype='<U9')
Coordinates:
    run      <U9 'r10i1p5f1

In [11]:
xr_output

<xarray.DataArray 'thetao' (run: 16, lag: 23, lev: 38, lat: 180)>
array([[[[            nan,             nan,             nan, ...,
           1.21328339e-01,  1.17280762e-01,             nan],
         [            nan,             nan,             nan, ...,
           1.21538989e-01,  1.17412649e-01,             nan],
         [            nan,             nan,             nan, ...,
           1.21742118e-01,  1.17826238e-01,             nan],
         ...,
         [            nan,             nan,             nan, ...,
          -2.93518667e-01, -2.60123697e-01,             nan],
         [            nan,             nan,             nan, ...,
          -3.10835265e-01, -2.68142546e-01,             nan],
         [            nan,             nan,             nan, ...,
          -3.13171874e-01, -2.58387583e-01,             nan]],

        [[            nan,             nan,             nan, ...,
           1.88938694e-01,  1.98417435e-01,             nan],
         [            nan,             nan,             nan, ...,
           1.88803344e-01,  1.97810412e-01,             nan],
         [            nan,             nan,             nan, ...,
           1.88340743e-01,  1.98059817e-01,             nan],
...
         [            nan,             nan,             nan, ...,
          -3.61061923e-01, -3.38792348e-01,             nan],
         [            nan,             nan,             nan, ...,
          -3.37978403e-01, -3.03270551e-01,             nan],
         [            nan,             nan,             nan, ...,
          -3.09582660e-01, -2.55834325e-01,             nan]],

        [[            nan,             nan,             nan, ...,
           3.86547416e-01,  3.53475142e-01,             nan],
         [            nan,             nan,             nan, ...,
           3.86166383e-01,  3.52834231e-01,             nan],
         [            nan,             nan,             nan, ...,
           3.86382294e-01,  3.53013071e-01,             nan],
         ...,
         [            nan,             nan,             nan, ...,
          -3.82856423e-01, -3.73628781e-01,             nan],
         [            nan,             nan,             nan, ...,
          -3.60319094e-01, -3.38237191e-01,             nan],
         [            nan,             nan,             nan, ...,
          -3.30949514e-01, -2.88876088e-01,             nan]]]])
Coordinates:
  * lev      (lev) float64 1.556 2.668 3.856 5.14 ... 333.9 370.7 411.8 457.6
  * run      (run) <U9 'r1i1p5f1' 'r2i1p5f1' ... 'r15i1p5f1' 'r16i1p5f1'
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
    lon      float64 270.5
  * lag      (lag) int64 -3 -2 -1 0 1 2 3 4 5 6 ... 11 12 13 14 15 16 17 18 19

In [12]:
average = xr_output.mean("run")
average

<xarray.DataArray 'thetao' (lag: 23, lev: 38, lat: 180)>
array([[[        nan,         nan,         nan, ...,  0.17686279,
          0.16354201,         nan],
        [        nan,         nan,         nan, ...,  0.1765183 ,
          0.16302859,         nan],
        [        nan,         nan,         nan, ...,  0.17618688,
          0.16260517,         nan],
        ...,
        [        nan,         nan,         nan, ..., -0.11164884,
         -0.09920575,         nan],
        [        nan,         nan,         nan, ..., -0.09509049,
         -0.08427494,         nan],
        [        nan,         nan,         nan, ..., -0.07785924,
         -0.06753417,         nan]],

       [[        nan,         nan,         nan, ...,  0.20473023,
          0.19424479,         nan],
        [        nan,         nan,         nan, ...,  0.20426084,
          0.19391929,         nan],
        [        nan,         nan,         nan, ...,  0.20386305,
          0.19338231,         nan],
...
        [        nan,         nan,         nan, ..., -0.04703232,
         -0.03085145,         nan],
        [        nan,         nan,         nan, ..., -0.04034924,
         -0.0227529 ,         nan],
        [        nan,         nan,         nan, ..., -0.03400916,
         -0.01505846,         nan]],

       [[        nan,         nan,         nan, ...,  0.1482768 ,
          0.14240487,         nan],
        [        nan,         nan,         nan, ...,  0.14766175,
          0.1417963 ,         nan],
        [        nan,         nan,         nan, ...,  0.14712024,
          0.14118072,         nan],
        ...,
        [        nan,         nan,         nan, ..., -0.04324012,
         -0.02613787,         nan],
        [        nan,         nan,         nan, ..., -0.03592685,
         -0.01751059,         nan],
        [        nan,         nan,         nan, ..., -0.02951592,
         -0.01042222,         nan]]])
Coordinates:
  * lev      (lev) float64 1.556 2.668 3.856 5.14 ... 333.9 370.7 411.8 457.6
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
    lon      float64 270.5
  * lag      (lag) int64 -3 -2 -1 0 1 2 3 4 5 6 ... 11 12 13 14 15 16 17 18 19

In [13]:
# Define output file path
output_path = "/nobackup/users/ommeren/ECEarth/150_sec_avg_corr.nc"

# Save as NetCDF
average.to_netcdf(output_path)

print(f"File saved successfully at: {output_path}")


File saved successfully at: /nobackup/users/ommeren/ECEarth/270_sec_avg_corr.nc


# -------------------------------------------------------------------------------